In [ ]:
from keras.models import model_from_json
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import KFold
import numpy as np

import util
import time

json_file = open('mnist_20161211_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("mnist_20161211_weights.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [ ]:
mnist = fetch_mldata('MNIST original')

data = mnist.data.reshape((len(mnist.data), 1, 28, 28))
np.random.shuffle(data)
data = data[:1000]
data = data.astype('float32')
data = data / 255

In [ ]:
kf = KFold(n_splits=10, shuffle=True)
print(data.shape)

for model_name in ['cnn']:#, 'cnn']:
    accuracies = []
    fold_index = 1
    predictions = []
    targets = []
    for train_index, test_index in kf.split(data):
        from keras.utils import np_utils
        nb_classes = 10
        batch_size = 128
        nb_epoch = 10
        x_train, x_test = data[train_index], data[test_index]
        y_train, y_test = mnist.target.astype('int')[train_index], mnist.target.astype('int')[test_index]
#         y_train = np_utils.to_categorical(y_train, nb_classes)
#         y_test = np_utils.to_categorical(y_test, nb_classes)
        clf, prediction = util.train_model(model_name, x_train, y_train, x_test)

        print(prediction[0])
        prediction_int = list(map(int, np.round(prediction)))
        predictions.append(prediction)
        targets.append(y_test)
#         conf_matrix = confusion_matrix(true_y, prediction_int)
#         title = "%s fold %d" % (model_name.upper(), fold_index)
#         plot_confusion_matrix(conf_matrix, ['not skin', 'skin'], title=title)

        accuracy = np.sum(np.diag(conf_matrix)) / np.sum(conf_matrix)
        accuracies.append(accuracy)
    #     print(accuracy)
    #     y = list(map(int, np.ravel(train_data[:, -1])))
    #     print(np.bincount(y))
    #     y = list(map(int, np.ravel(test_data[:, -1])))
    #     print(np.bincount(y))
        fold_index += 1
        pass

    conf_matrix = confusion_matrix(targets, predictions)
    title = "Confusion matrix %s" % (model_name.upper())
    plot_confusion_matrix(conf_matrix, list(range(10)), title=title)
    print("\nCross validated accuracy: %f\n\n" % np.mean(accuracies))

    pass

In [ ]:
conf_mat = np.array(
[[ 2236 ,    0 ,    4 ,    0 ,    1 ,    3 ,    2 ,    2 ,    5 ,    3 ],
 [          0 , 2553 ,    7 ,    1 ,    2 ,    1 ,    0 ,    4 ,    2 ,    1 ],
 [          4 ,    3 , 2239 ,    6 ,    6 ,    0 ,    1 ,   17 ,    5 ,    2 ],
 [          3 ,    1 ,    6 , 2398 ,    0 ,   29 ,    0 ,    4 ,    9 ,    4 ],
 [          3 ,    3 ,    1 ,    0 , 2207 ,    0 ,    3 ,    5 ,    1 ,   11 ],
 [          3 ,    0 ,    1 ,    8 ,    3 , 2092 ,    5 ,    0 ,    9 ,    5 ],
 [          6 ,    5 ,    0 ,    0 ,    3 ,    7 , 2229 ,    0 ,    4 ,    0 ],
 [          1 ,    3 ,    6 ,    2 ,    3 ,    2 ,    0 , 2391 ,    1 ,    5 ],
 [          3 ,    4 ,    5 ,    6 ,    4 ,    9 ,    3 ,    1 , 2213 ,    7 ],
 [          3 ,    1 ,    0 ,    3 ,   14 ,    1 ,    0 ,   10 ,   11 , 2210 ]])
from matplotlib import pyplot as plt

plt.figure(figsize=(10, 7))
util.plot_confusion_matrix(cm=conf_mat, classes=list(range(10)), save_plot=False, title="confusion matrix CNN")

In [ ]:
conf_mat = np.array(
[[2223,    0,    0,    5,    3,    6,    8,    1,    8,    2],
 [  43, 2473,   14,    8,    1,    5,    2,    4,   19,    2],
 [ 101,    6, 2030,   19,   29,    4,   16,   27,   46,    5],
 [  62,    1,    9, 2268,    0,   57,    2,    9,   36,   10],
 [  47,    4,    5,    0, 2132,    0,    4,    1,    6,   35],
 [  49,    4,    1,   21,    5, 2005,   13,    1,   14,   13],
 [  45,    4,    0,    0,   13,   19, 2168,    0,    5,    0],
 [  80,    2,    9,    4,   19,    1,    0, 2235,    8,   56],
 [  59,    6,    4,   24,    5,   30,    7,    1, 2110,    9],
 [  81,    1,    0,   12,   27,    4,    0,   22,   20, 2086]])
from matplotlib import pyplot as plt

plt.figure(figsize=(10, 7))
util.plot_confusion_matrix(cm=conf_mat, classes=list(range(10)), save_plot=False, title="confusion matrix SVM")


In [ ]:

plt.show()